# Introduction to Requests: Types, Responses, Use Cases


It is very often useful to be able to communicate between computers and services in our code

To do this, we use requests!

HTTP requests are used by one piece of code to get or send along pieces of information from another service or computer.


## HTTP

Early on in the semester, we covered some of the basics of HTTP.

It is a request/response system that allows a client to send requests to a server and get a response using a TCP handshake for verification.

One example here is a web browser. Every time that you visit a website, you send a request to the server of that site and recieve an HTML page as a response, which your browser renders

## Python Requests Library


Today, we'll be covering the python requests library, which can be installed with 'pip install requests' in the command line and then the following import statement in your code:

In [2]:
import requests

The requests library will allow us to send HTTP requests to other servers or services and yielding a python object that contains a response code, a header, and a body

## Request Methods

- GET:
    - Used to grab data from a website or service
    - This is what browsers use to access a website
    - You can also pass some key-value pairs for authentication and other arguments (google searches use a get request)
    - The following is an example of a get request to google.com (we get a resposne with status code 200, meaning that we successfully connected to the server)
    - If you were to print r.text here, you would get a large string of HTML

In [4]:
r = requests.get('https://www.google.com')

<Response [200]>

- POST
    - Used to send a collection of data to a site or service
    - We pass in a dictionary to the function call to send this data with our request
    - POST requests are often used by websites to send form data that the user inputs to the server to be processed.
    - The following is an example of a POST request to a demo URL passing in a dictionary

In [14]:
url = 'https://www.w3schools.com/python/demopage.php'
myobj = {'somekey': 'somevalue'}

x = requests.post(url, json = myobj)
x

200

- Other request methods supported by the Requests module (less common methods):
    - DELETE:
        - Used to send a request that indicates deletion of something on the server
    - PATCH:
        - A set of instructions to modify the server
    - HEAD:
        - Requests just the headers of a response to a get request from the server
    - PUT:
        - Replace a target resource with payload

- Request methods not supported by the requests module
    - CONNECT:
        - Establish a two-way connection stream between server and client
    - OPTIONS:
        - Requests the allowed options for connection to a server
    - TRACE:
        - Traces the forwarding involved in a request

## Response Codes


- 100 -199
    - These status codes provide information about the request. Often, this means that the server is still loading or doing some sort of internal work
- 200 - 299
    - A 200 status code means that the request was successful (the server exists and the provided inputs are sufficient)
    - The other codes in this range mean that the request was successfully sent, but they often indicate other important pieces of information about the response
- 300 - 399
    - These are various redirection codes. They indicate that the user is being directed away from the server initially requested
- 400 - 499
    - These are unsuccessful requests, the most common of which are:
        - 400: Bad request, meaning the request format is not correct
        - 401: Unauthorized, client is not authorized to make this request
        - 404: Not Found, the requested resource cannot be found
- 500 - 599
    - 500 indicates an internal server error
    - The rest of these mean that something is going wrong in the server

If you are looking for an exhaustive list of the status codes which I have not listed here, you can find it on https://developer.mozilla.org/en-US/docs/Web/HTTP/Status

You can see the status code of a request response in python by accessing the status_code property of the response as follows:

In [15]:
url = 'https://www.w3schools.com/python/demopage.php'
myobj = {'somekey': 'somevalue'}

x = requests.post(url, json = myobj)
x.status_code

200

## When might we use requests?

There are various reasons that we might want to send requests in our python code.

Programmatically accessing resources on the internet can be very valuable!

#### Webscraping

Suppose that there is a website, or list of websites, which has a dataset or datasets that we are interested in downloading

We could certainly manually visit each site and copy the values into CSV files, but the requests library would provide us with a much quicker solution which we could run periodically to automatically update our dataset

In the following code, we grab the data file from https://dasl.datadescription.com/download/data/3176 and save it to outfile.csv

In [27]:
r = requests.get('https://dasl.datadescription.com/download/data/3176')


with open('outfile.csv', 'w', encoding="utf-8") as f:
    f.write(r.text)

In some cases, the data we want might not be available for download from the website, so we can use tools like BeautifulSoup4 to grab the HTML and then parse through the HTML to isolate the data that we are interested in. This is the subject of another lecture, so we will not get into it yet.

### Requesting data from APIs on the internet


There are many resources on the internet which are designed to be accessed programmatically. The use cases of these services varies from providing small amounts of data to running complex functions that have been implemented by the service on your inputs.

Let's explore a very simple use case. The https://catfact.ninja/fact website will provide the user with a fact about cats on each call. We can use the python requests module to get an example of one as follows:

In [37]:
r = requests.get('https://catfact.ninja/fact')

print(r.text)

{'fact': 'Cats can predict earthquakes. We humans are not 100% sure how they do it. There are several different theories.', 'length': 111}


This clearly looks just like a dictionary, so we can use the response.json() method to cast the text into a dictionary, then access the fact itself.

In [39]:
r = requests.get('https://catfact.ninja/fact')

print(r.json())

print(r.json()['fact'])

{'fact': 'Many Egyptians worshipped the goddess Bast, who had a woman’s body and a cat’s head.', 'length': 84}
Many Egyptians worshipped the goddess Bast, who had a woman’s body and a cat’s head.


As an exercise, use a for loop to print 10 cat facts from the API:

In [ ]:
### Write code here 




### Attacking a website

I'm not saying that you should do this, but using some multithreading and an infinite loop with the requests module, you could try running a Denial of Service attack on smaller websites by sending a significant number of concurrent reqeusts to a site to the point that the site is so busy processing these requests that it cannot handle any other user traffic.

Most big websites will have protection against these kinds of things that make it very difficult to actually take down their site, but the idea here is that sending requests with python can be very powerful

### Exercises!


1. Use the condesk API at https://api.coindesk.com/v1/bpi/currentprice.json to get the current price of bitcoin!

In [41]:
### Code for exercise 1

2. Get the HTML from https://www.oberlin.edu/ and store it in a file locally

In [ ]:
### Code for exercise 2

3. Use the api at https://dog.ceo/api/breeds/image/random to get the url for an image of a dog, then send a get request to the url and save the image locally

In [ ]:
### Code for exercise 3